
<div align="center">
  <h1></h1>
  <h1>Multi-Agent Meal Planning</h1>
  <h4 align="center"></h4>
</div>


In this notebook, I will design and implement a Multi-Agent Meal Planning System that generates meal recommendations while adhering to budgetary and nutritional constraints. Using a team of specialized agents, I'll create a dynamic and interactive system capable of tailoring meal plans to diverse user requirements.


# 1. Access to GROQ.
Execute the following cell to connect to your Groq account.

In [1]:
import os
from getpass import getpass

tokenGROQ = getpass('Enter GROQ_API_KEY here: ')
os.environ["GROQ_API_KEY"] = tokenGROQ

Enter GROQ_API_KEY here:  ········


# 2. Packages
Execute the following code cells for installing the packages needed.

In [2]:
!pip install autogen-agentchat "autogen-ext[openai]" matplotlib yfinance -q
!pip install groq -q

# 3. Problem Statement: Multi-Agent Meal Planning System with Budget Management


In this notebook we will design and implement a multi-agent system to assist users in meal planning while addressing the following:

* Dietary preferences: Personalization based on restrictions and user preferences.
* Nutritional goals: Balanced and healthy meal recommendations.
* Financial constraints: Ensuring affordability through budget validation.

This system will include a Budget Checker Tool to guarantee all recommendations fit within the user's financial limits.


**System**

1. Personalized Meal Recommendations:

* Provides tailored meal suggestions for breakfast, lunch, dinner, and snacks.
* Considers dietary restrictions, time availability, and individual preferences.
2. Budget Validation:

* Uses the Budget Checker Tool to confirm that meal costs fit within the user’s budget.
3. User-Friendly Assistance:

* Delivers clear, concise, and actionable recommendations for each meal type.

**Agent Roles**

1. Breakfast Agent:

Role: Recommends breakfast options that align with dietary preferences, preparation time, and nutritional goals.

Specialization: Provides a single breakfast suggestion and validates its affordability using the Budget Checker Tool.
2. Lunch Agent:

Role: Suggests balanced and nutritious lunch options tailored to dietary needs.

Specialization: Offers one lunch suggestion and ensures it fits within the budget.

3. Dinner Agent:

Role: Recommends practical dinner options based on dietary requirements, budget, and preparation constraints.

Specialization: Validates that dinner suggestions are both feasible and affordable.

4. Snack Agent:

Role: Proposes healthy and satisfying snack options to complement the day’s meals.

Specialization: Verifies that snack recommendations are affordable and nutritionally balanced.

5. Budget Agent:

Role: Ensures all meal costs remain within the user's remaining budget by using the Budget Checker Tool.

Specialization: Provides real-time budget feedback to guide meal selections and prevent overspending.



**Custom Tool: Budget Checker Tool**
1. Purpose:

* Validates if a proposed meal cost fits within the user’s current budget.

2. Features:

* Budget Validation: Compares the cost of a meal against the remaining budget.
* Feedback: Provides clear, actionable messages indicating:

  Whether the cost is affordable.

  How much budget remains or is exceeded.

* Integration: Used by all meal-specific agents to ensure recommendations are financially feasible.

# 4. Configure the Model Client:

Use the OpenAIChatCompletionClient class to create an instance of the model client.

Configure it with the following:

* Model name: "llama3-70b-8192".

* API base URL: "https://api.groq.com/openai/v1".

* API key: Use an environment variable named "GROQ_API_KEY" for secure authentication.

* Model settings: Include attributes like whether vision is enabled, function-calling support, and JSON output format.

In [3]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Step 1: Create the model client
# Complete the following implementation
model_client = OpenAIChatCompletionClient(
    model="llama3-70b-8192",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
        "temperature" : 0.7,
    },
)

# Step 2: Verify the client is set up correctly
print(model_client)

# 5. Budget Tracker Tool

The provided code implements a Budget Checker Tool to validate whether a given cost is within the current budget.

In [4]:
from autogen_core.tools import FunctionTool

def budget_checker_tool(cost: float, current_budget: float) -> dict:
    """
    Check if a given cost fits within the current budget.

    :param cost: float, the cost to check.
    :param current_budget: float, the current remaining budget.
    :return: dict, containing whether the cost fits the budget and a message.
    """
    if cost > current_budget:
        return {
            "fits_budget": False,
            "message": f"This exceeds your budget by ${cost - current_budget:.2f}."
        }
    return {
        "fits_budget": True,
        "message": f"The cost of ${cost:.2f} fits within your budget."
    }

# Example usage with the FunctionTool
budget_checker_tool = FunctionTool(
    budget_checker_tool,
    description="Check if a given cost fits within the current budget."
)

# 6. Agent Design

In this task, I will implement a set of specialized Assistant Agents to collaboratively plan meals while addressing dietary, nutritional, and budgetary constraints. I will design Meal Planner Agents for breakfast, lunch, dinner, and snacks, each with clear roles and behavioral instructions.

In [5]:
# Import the required class
from autogen_agentchat.agents import AssistantAgent

# Step 1: Define the Budget Agent
# Complete the following implementation
budget_agent = AssistantAgent(
    name="Budgeting_agent",
    tools=[budget_checker_tool],  # Use the budget checker tool
    model_client=model_client,
    description="Agent is responsible for keeping budget in check.",
    system_message="""You are an able budget planning agent. Your job is to use the provided tool to check if budget stays in check
    and provide an appropriate response. Collect all the meal costs from various agents, add the cost to single number, then use the tool to check if budget is within limit and
    then provide a clear response. If total meal cost is within limit, then ask agents to summarise interactions. If meal is beyond limit
    then ask agents to make changes to their recommendations. Total budget should be under 10 Euros."""
)

# Step 2: Define the Breakfast Planner Agent
# Complete the following implementation
breakfast_agent = AssistantAgent(
    name="Breakfast_agent",
    model_client=model_client,
    description="An agent that provides breakfast recommendations.",
    system_message="""You are an intelligent breakfast recommender in a team of five agents with respective roles. Just recommend one BREAKFAST option and give its cost. Breakfast option should address given dietary
    and nutritional constraints. Also you have to give price for your breakfast recommendation. Only consider budget feedback if available. Reply only to budget agent."""
)

# Step 3: Define the Lunch Planner Agent
# Complete the following implementation
lunch_agent = AssistantAgent(
    name="Lunch_agent",
    model_client=model_client,
    description="An agent that provides lunch recommendations.",
    system_message="""You are an intelligent lunch recommender in a team of five agents with respective roles. Just recommend one LUNCH option and give its cost. Lunch option should address given dietary
    and nutritional constraints. Also you have to give price for your lunch recommendation. Only consider budget feedback if available. Reply only to budget agent."""
)

# Step 4: Define the Dinner Planner Agent
# Complete the following implementation
dinner_agent = AssistantAgent(
    name="Dinner_agent",
    model_client=model_client,
    description="An agent that provides dinner recommendations.",
    system_message="""You are an intelligent dinner recommender in a team of five agents with respective roles.  Just recommend one DINNER and give its cost. Dinner option should address given dietary
    and nutritional constraints. Also you have to give price for your dinner recommendation. Only consider budget feedback if available. Reply only to budget agent."""
)

# Step 5: Define the Snack Planner Agent
# Complete the following implementation
snack_agent = AssistantAgent(
    name="Snack_agent",
    model_client=model_client,
    description="An agent that provides snack recommendations.",
    system_message="""You are an intelligent snack recommender in a team of five agents with respective roles. Just recommend one SNACK and give its cost. Snack option should address given dietary
    and nutritional constraints. Also you have to give price for your snack recommendation. Only consider budget feedback if available. Reply only to budget agent."""
)

# Step 6: Test the agents
print("Agents initialized successfully!")


Agents initialized successfully!


# 7. Implementing Termination
Implement a MaxMessageTermination with 10 max_messages

In [6]:
from autogen_agentchat.conditions import MaxMessageTermination
# Complete the following implementation
termination = MaxMessageTermination(10)

# 8. RoundRobinGroupChat

In this part, I will implement a RoundRobinGroupChat team to coordinate multiple agents for a meal-planning task. The system will use a sequential approach, ensuring that each agent contributes to the task in a pre-defined order.


In [7]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
# Complete the following implementation
team = RoundRobinGroupChat([breakfast_agent, lunch_agent, dinner_agent, snack_agent, budget_agent], termination_condition=termination)

In [8]:
# Complete the following implementation
await Console(team.run_stream(task="""Give me respective food recommendations. Nutrition wise : meal should have more high protein. Dietary : vegan. Mention prices for each meal."""))

---------- user ----------
Give me respective food recommendations. Nutrition wise : meal should have more high protein. Dietary : vegan. Mention prices for each meal.
---------- Breakfast_agent ----------
As the Breakfast Agent, I'm happy to recommend a high-protein, vegan breakfast option that meets your dietary and nutritional constraints.

**Recommended Breakfast Option:** Tofu Scramble with Quinoa and Roasted Vegetables

This breakfast dish is packed with protein-rich tofu, quinoa, and roasted vegetables. Here's a breakdown of the ingredients:

* 1/2 cup cooked quinoa (8g protein)
* 1/2 cup scrambled tofu (20g protein)
* 1 cup mixed roasted vegetables (broccoli, bell peppers, carrots, onions)
* 1 tablespoon olive oil
* Salt and pepper to taste
* Optional: 1 tablespoon nutritional yeast for an extra cheesy flavor

**Price:** This breakfast option costs approximately $3.50 per serving.

The quinoa provides a complete protein, while the tofu adds an extra boost of protein. This meal 

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Give me respective food recommendations. Nutrition wise : meal should have more high protein. Dietary : vegan. Mention prices for each meal.', type='TextMessage'), TextMessage(source='Breakfast_agent', models_usage=RequestUsage(prompt_tokens=105, completion_tokens=200), content="As the Breakfast Agent, I'm happy to recommend a high-protein, vegan breakfast option that meets your dietary and nutritional constraints.\n\n**Recommended Breakfast Option:** Tofu Scramble with Quinoa and Roasted Vegetables\n\nThis breakfast dish is packed with protein-rich tofu, quinoa, and roasted vegetables. Here's a breakdown of the ingredients:\n\n* 1/2 cup cooked quinoa (8g protein)\n* 1/2 cup scrambled tofu (20g protein)\n* 1 cup mixed roasted vegetables (broccoli, bell peppers, carrots, onions)\n* 1 tablespoon olive oil\n* Salt and pepper to taste\n* Optional: 1 tablespoon nutritional yeast for an extra cheesy flavor\n\n**Price

Expected output should show the following:

**Agents' Initial Suggestions:** Each agent provides meal recommendations (e.g., breakfast, lunch, dinner, snack) with corresponding costs.

**Budget Validation:** The Budget_Agent uses the budget_checker_tool to validate if the total cost fits within the budget. If it doesn't, the agent provides a clear message explaining how much the budget is exceeded.

**Adjustments by Agents:** If the budget is exceeded, agents revise their recommendations to bring the total cost within the budget.

**Task Completion:** The system stops after reaching the maximum number of allowed messages, with a summary of the interactions.